<p style="text-align:left;color:red;font-weight:bold;font-size:16pt;padding-bottom:20px">Please, copy this notebook before editing!</p>

# Homework 10

## Hyper-parameter Tuning

In this exercise, you experiment with hyper-parameter tuning. You use XGBoost 

Examples are takeng from this blog post
https://towardsdatascience.com/xgboost-fine-tune-and-optimize-your-model-23d996fab663


## Data Set

| Data Set Characteristics | |
|-|-|
| Number of Instances | 20,640 |
| Number of Attributes| 8 numeric, predictive attributes and the target |
| Missing Attribute Values | None | 


| Attribute | Information |
|-|-|
| `MedInc` | median income in block group |
| `HouseAge` | median house age in block group |
| `AveRooms` | average number of rooms per household |
| `AveBedrms` | average number of bedrooms per household |
| `Population` | block group population |
| `AveOccup` | average number of household members |
| `Latitude` | block group latitude |
| `Longitude` | block group longitude |


This dataset was obtained from the StatLib repository. https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The **target variable** `MedHouseVal` is the median house value for California districts, expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per census block group. A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people).

A household is a group of people residing within a home. Since the average number of rooms and bedrooms in this dataset are provided per household, these columns may take surprisingly large values for block groups with few households and many empty houses, such as vacation resorts.


#### References

Pace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions, Statistics and Probability Letters, 33 (1997) 291-297



### 10.1 Split Data Set
Write a function `split_dataframe` that splits a given DataFrame randomly into two sets. The second argument of the function is the `sample_ratio`, that determines the size of the first set. E.g. `sample_ratio` = 0.8 means that 80% of the data go into the first set, and 20% into the second. The functions returns a tuple of two DataFrames.

One way to split randomly:
1. create a (NumPy) vector of random values between 0 and 1. The vector is of the length of the data set.
2. create a boolean vector by comparing the vector elements to the `sample_ratio`. E.g. `mask = (np.random.rand(df.shape[0]) <= sample_ratio)`
3. use `mask` to select the first data set, and the negation `~mask` for the second.
4. returns copies of the new dataframes (with `.copy()`) so that you can easily modify the resulting dataframes.

In [ ]:
# from sklearn.datasets import fetch_california_housing
from typing import Tuple
import pandas as pd
import numpy as np

housing_df = pd.read_csv('/data/IFI8410/california_housing/california_housing.csv')
print(f"Number of records: {housing_df.shape[0]:,}")
housing_df.dtypes

Save your solution to `solution_10_1.py`

In [ ]:
%%writefile solution_10_1.py

from typing import Tuple
import pandas as pd
import numpy as np

def split_dataframe(df: pd.DataFrame, sample_ratio: float = 0.8) -> Tuple[pd.DataFrame, pd.DataFrame]:
    ### your code here










    

In [ ]:
# Try your code
from solution_10_1 import split_dataframe
import pandas as pd

housing_df = pd.read_csv('/data/IFI8410/california_housing/california_housing.csv')
print(f"Number of records: {housing_df.shape[0]:,}")

housing_train, housing_test = split_dataframe(housing_df, 0.7)
del housing_df 
print(f"Number of records in training set: {housing_train.shape[0]:,}")
print(f"Number of records in test set: {housing_test.shape[0]:,}")


#### Test 10.1 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 1

### 10.2 Problem Statement
It is important that the training and test sets have the same value distribution. The function `compare_feature_distributions` computes the relative differences in the statistical disctributions of the two data sets for each feature.

The resulting table should show values very close to 0. Specifically for the mean, standard deviation, and the quartiles.

In [ ]:
def compare_feature_distributions(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    # Create a joined table of the "descriptions" of each table
    # Note: we use the transposed DataFrame, so that the features become row indices, and the metrics become columns
    comp = df1.describe().T.join(df2.describe().T, lsuffix='_one', rsuffix='_two')
    mets = ['mean', 'std', 'min', '25%', '50%', '75%', 'max']
    # Calculate the value range for each feature
    comp['delta'] = comp.apply(lambda r: max(r.max_one, r.max_two) - min(r.min_one, r.min_two), axis=1)
    for met in mets:
        # Calculate the relative difference for each metric
        comp[f"diff_{met}"] = (comp[f"{met}_one"] - comp[f"{met}_two"])/comp['delta']
    res_cols = [ f"diff_{met}" for met in mets ]
    # return a new copy of the subset of the DataFrame
    return comp[res_cols].copy()
    

Try is out on your train and test data sets.

In [ ]:
cmp_df = compare_feature_distributions(housing_train, housing_test)
# save the comparison
cmp_df.to_csv('output_10_2.csv', index_label='feature')
display(cmp_df)

#### Test 10.2 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 2

### 10.3 Data Pre-Processing
Implement a pre-processor (aka `ColumnTransformer`) to apply the standard scaler to the feature columns, **but not** the target feature `MedHouseVal`.
The pre-processor that the function returns must be already trained, aka `.fit()` (on the training set)

Use the example in Homework 7 to build a pre-processor:
- Create an instance of `StandardScaler`
- Create a pipeline object (`Pipeline`) with the standard scaler as step
- Create a list of numerical feature columns without the target feature `MedHouseVal`. (You can extract them from the dataframe)
- Create a `ColumnTransformer` object with the pipeline for the numeric columns, use `remainder="passthrough"` to bypass all other columns

Save your solution to `solution_10_3.py`

In [ ]:
%%writefile solution_10_3.py

from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline

def build_preprocessor(training_df, target_feature: str = 'MedHouseVal') -> ColumnTransformer:
    ### your code here
    






    


In [ ]:
# Try your code

from solution_10_1 import split_dataframe
from solution_10_3 import build_preprocessor
import pandas as pd

df = pd.read_csv('/data/IFI8410/california_housing/california_housing.csv')
df_train, df_test = split_dataframe(df, 0.6)

# Create pre-processor
pre_processor = build_preprocessor(df_train, target_feature='MedHouseVal')

# Check distribution
print("Training Set")
Xy_train = pre_processor.transform(df_train)
display(Xy_train.mean(axis=0))
display(Xy_train.std(axis=0))

print("Test Set")
Xy_test = pre_processor.transform(df_test)
display(Xy_test.mean(axis=0))
display(Xy_test.std(axis=0))




#### Test 10.3 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 3

### 10.4 Train Regression Model

Use [XGBoost](https://xgboost.readthedocs.io/en/stable/) for to build a regression model.

XGBoost is a popular algorithm that can be used for regression and classification.

This article gives a nice introduction to XGBoost: <https://towardsdatascience.com/xgboost-fine-tune-and-optimize-your-model-23d996fab663>

Execute the cells below to see how it's used.

In [ ]:
from solution_10_1 import split_dataframe
from solution_10_3 import build_preprocessor
import pandas as pd
import xgboost as xgb
import datetime

df = pd.read_csv('/data/IFI8410/california_housing/california_housing.csv')
# Let's split the data 80:20
df_train, df_test = split_dataframe(df, 0.8)

# Create pre-processor
pre_processor = build_preprocessor(df_train, target_feature='MedHouseVal')

The following steps process the training data and package them for model training.
XGBoost is not part of the Scikit-Learn package. So, some if the data structures are different. Specifically, the Python XGBoost package has its own structure `DMatrix`.

In [ ]:
Xy_train = pre_processor.transform(df_train)
X = Xy_train[:, :-1]
y = Xy_train[:, -1]
print(X.shape, y.shape)
dmatrix = xgb.DMatrix(data=X, label=y)
print(f"Number of rows: {dmatrix.num_row()}, number of columns: {dmatrix.num_col()}")

There are a number of hyper-parametes. Check the [documentation](https://xgboost.readthedocs.io/en/stable/) for details.

In [ ]:
params={ 'objective':'reg:squarederror',
         'max_depth': 6, 
         'colsample_bylevel':0.5,
         'learning_rate':0.01,
         'random_state':20}

# Train model
model_xgb = xgb.train(dtrain=dmatrix, params=params, num_boost_round=1000)

# Save model
model_xgb.save_model('model_10_4.json')

Evaluate the model on the test data

In [ ]:
Xy_test = pre_processor.transform(df_test)
X_test = Xy_test[:, :-1]
y_test = Xy_test[:, -1]
dmatrix_test = xgb.DMatrix(data=X_test)
print(f"Number of rows: {dmatrix_test.num_row()}, number of columns: {dmatrix_test.num_col()}")

y_hat = model_xgb.predict(dmatrix_test)
RSME = np.sqrt(np.square(y_hat - y_test).mean())
print(f"RMSE: {RSME:.2f}")

# Add predicted values to test dataframe and save
df_test['Predicted_MedHouseVal'] = y_hat
df_test.to_csv('output_10_4.csv', index=None)

#### Test 10.4 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 4

### 10.5 Hyper-parameter Optimization

You need to experiment with different parameters to improve the model. The following shows an example of the parameter space:

In [ ]:
param_space = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': [0.01, 0.1, 0.2, 0.3],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [100, 500, 1000]}
n_params = np.array([ len(v) for k,v in param_space.items() ]).prod()
print(f"Number of parameter combinations: {n_params:,}")

Use a randomize search algorithm to experiment with different paramer sets.

**This will take a while!**

In [ ]:
xgbr = xgb.XGBRegressor(seed = 20)
clf = RandomizedSearchCV(estimator=xgbr,
                         param_distributions=param_space,
                         scoring='neg_mean_squared_error',
                         n_iter=25,
                         verbose=1)
T_0 = datetime.datetime.now()

clf.fit(X, y)

print(f"Elapsed time: {datetime.datetime.now()-T_0}")
print("Best parameters:", clf.best_params_)

# Save parametes
json.dump(clf.best_params_, open('output_10_5.json', 'w', encoding='utf-8'), indent=4)

# Evaluate
y_hat2 = clf.predict(X_test)
RMSE_2 = np.sqrt(np.square(y_hat2 - y_test).mean())
print(f"Best RMSE: {RMSE_2:.2f}")

#### Test 10.5 Execute the cell below to test your solution...

In [ ]:
! test/run_test.sh 5

# Testing

Execute the cell below to run all tests...

In [ ]:
! test/run_test.sh

# Homework Submission
- This homework is due by 2024-04-26, 5:30PM (EDT)
- Make sure that all your programs and output files are in the exact folder as specified in the instructions.
- All file names on this system are case sensitive. Verify if you copy your work from a local computer to your home directory on ARC.

**Run the cell below to submit your work.**

- You may submit your work multipe times up to the deadline of the assignment.
- Please note that any files that you previously submitted will
be overwritten by your current files.

In [ ]:
! ./submit.sh -y